In [1]:
import import_ipynb
import random
import json
from nltk_utils import tokenize, stem ,bag_of_words
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from model import NeuralNet

importing Jupyter notebook from nltk_utils.ipynb
importing Jupyter notebook from model.ipynb


In [2]:
with open('intents.json','r') as f:
    intents=json.load(f)


In [3]:
all_words=[]
tags=[]
xy=[]
for intent in intents['intents']:
    tag=intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:  
        w=tokenize(pattern)
        all_words.extend(w)
        xy.append((w,tag))
ignore_words=['!','@','#','$','&','*','(',')','-','=','?','.',',']
all_words=[stem(w) for w in all_words if w not in ignore_words]

In [4]:
all_words=sorted(set(all_words))


In [5]:
tags=sorted(set(tags))
tags

['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']

In [6]:
X_train=[]
y_train=[]
for pattern_sentence,tag in xy:
    bag=bag_of_words(pattern_sentence,all_words)
    X_train.append(bag)
    label=tags.index(tag)
    y_train.append(label)
X_train=np.array(X_train)
y_train=np.array(y_train)
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 1.]], dtype=float32)

In [7]:
class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples=len(X_train)
        self.x_data=X_train
        self.y_data=y_train
        
    def __getitem__(self,index):
        return self.x_data[index],self.y_data[index]
    def __len__(self):
        return self.n_samples

In [8]:
#Hyper parameter 
batch_size=8
hidden_size=8
output_size=len(tags)
input_size=len(X_train[0])
learning_rate=0.001
num_epochs=1000

In [9]:
dataset=ChatDataset()
trainloader=DataLoader(dataset=dataset,batch_size=batch_size,shuffle=True,num_workers=2)
device=torch.device('cpu')
model=NeuralNet(input_size,hidden_size,output_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
for epoch in range(num_epochs):
    for (words, labels) in trainloader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

/home/akaul/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/autograd/__init__.py:130: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729009598/work/c10/cuda/CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(


Epoch [100/1000], Loss: 1.3413
Epoch [200/1000], Loss: 1.1571
Epoch [300/1000], Loss: 0.0780
Epoch [400/1000], Loss: 0.0095
Epoch [500/1000], Loss: 0.0100
Epoch [600/1000], Loss: 0.0132
Epoch [700/1000], Loss: 0.0084
Epoch [800/1000], Loss: 0.0012
Epoch [900/1000], Loss: 0.0005
Epoch [1000/1000], Loss: 0.0018


In [11]:
data={
    "model_state":model.state_dict(),
    "output_size":output_size,
    "input_size":input_size,
    "hidden_size":hidden_size,
    "all_words":all_words,
    "tags":tags  
}

In [12]:
FILE="data.pth"
torch.save(data,FILE)
print(f,'training completed. File saved to {FILE}')

<_io.TextIOWrapper name='intents.json' mode='r' encoding='UTF-8'> training completed. File saved to {FILE}


In [13]:
'''import numpy as np
import random
import json
import import_ipynb
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from nltk_utils import bag_of_words, tokenize, stem
from model import NeuralNet

with open('intents.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

# Hyper-parameters 
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')

data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')'''

'import numpy as np\nimport random\nimport json\nimport import_ipynb\nimport torch\nimport torch.nn as nn\nfrom torch.utils.data import Dataset, DataLoader\n\nfrom nltk_utils import bag_of_words, tokenize, stem\nfrom model import NeuralNet\n\nwith open(\'intents.json\', \'r\') as f:\n    intents = json.load(f)\n\nall_words = []\ntags = []\nxy = []\n# loop through each sentence in our intents patterns\nfor intent in intents[\'intents\']:\n    tag = intent[\'tag\']\n    # add to tag list\n    tags.append(tag)\n    for pattern in intent[\'patterns\']:\n        # tokenize each word in the sentence\n        w = tokenize(pattern)\n        # add to our words list\n        all_words.extend(w)\n        # add to xy pair\n        xy.append((w, tag))\n\n# stem and lower each word\nignore_words = [\'?\', \'.\', \'!\']\nall_words = [stem(w) for w in all_words if w not in ignore_words]\n# remove duplicates and sort\nall_words = sorted(set(all_words))\ntags = sorted(set(tags))\n\nprint(len(xy), "patte